This notebook is used to scrape data from the hhs breach portal https://ocrportal.hhs.gov/ocr/breach/breach_report.jsf and populate the Breach Healthcare website.  www.breachhealthcare.com.  

The notebook uses the Selenium webdriver api to scrape data from the hhs portal.  The data is downloaded and then sent directly to a Google Sheet via the gspread api which is used as the source for a Tableau Public workbook.  Tableau Public enables a connection to Google Sheets enabling the workbooks to keep in sync with the Google Sheet.  

There are a couple of areas I'm still working on developing including moving the csv file download location to a cloud environment, other than my PC.  The second area of improvement is having the python script kickoff automatically each day rather than manually running it.

Before the automation, the process was fairly simple, but manual and time consuming.  The process was to go to the HHS portal site, download one csv file for Archived records, download another file for records Under Investigation, combine the two files and save.  Then open Tableau, refresh the existing tables and republish the scorecard.  

In [1]:
from selenium import webdriver
import time
import pandas as pd
import glob
import os

#this code is for the Selenim web driver to get files
browser = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')
url = 'https://ocrportal.hhs.gov/ocr/breach/breach_report.jsf'
browser.get(url) 

#This code downloads UI csv - the xpath was created by viewing the page source on the url to find the button pushed
browser.find_element_by_xpath('//*[@id="ocrForm:j_idt365"]').click()
time.sleep(5)
browser.close()

today = pd.Timestamp.today().strftime('%Y%m%d%H%M')
fileformat = '_UnderInvestigation_'
print(today)

#the Selenium web driver uses Chrome to download the file to my downloads folder
fd = r"C:\Users\ted\Downloads\breach_report.csv"

#To keep files organized, and maintaina history, the file is moved to a different folder and appended with the format and date
os.rename(fd,r'C:\Users\ted\OneDrive - Viztric\Clients\Shame Healthcare\BreachReport'+fileformat+today+'.csv') 

#Finds the most recent file in list to create a dataframe from
list_of_files = glob.glob(r'C:\Users\ted\OneDrive - Viztric\Clients\Shame Healthcare\*'+fileformat+'*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getmtime)
#print(list_of_files)
#print(latest_file)

#Creates a dataframe of Under Investigation records
df_ui = pd.read_csv(latest_file, parse_dates=['Breach Submission Date'])
#adds columns for Download data and FileType - the '_' is used to make a prettier file name, but stripped for the dataframe
df_ui['DownloadDate'] = today
df_ui['FileType'] = fileformat.strip('_')
#print(df_ui.head())

browser = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')
url = 'https://ocrportal.hhs.gov/ocr/breach/breach_report.jsf'
browser.get(url) 

#This code selects the "Archive" then downloads csv
browser.find_element_by_xpath('//*[@id="ocrForm:j_idt24"]').click()
time.sleep(5)
browser.find_element_by_xpath('//*[@id="ocrForm:j_idt365"]').click()
time.sleep(15)
browser.close()

today = pd.Timestamp.today().strftime('%Y%m%d%H%M')
fileformat = '_Archive_'
print(today)

#the Selenium web driver uses Chrome to download the file to my downloads folder
fd = r"C:\Users\ted\Downloads\breach_report.csv"

#To keep files organized, and maintaina history, the file is moved to a different folder and appended with the format and date
os.rename(fd,r'C:\Users\ted\OneDrive - Viztric\Clients\Shame Healthcare\BreachReport'+fileformat+today+'.csv') 


fileformat = '_Archive_'
list_of_files = glob.glob(r'C:\Users\ted\OneDrive - Viztric\Clients\Shame Healthcare\*'+fileformat+'*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getmtime)
#print(list_of_files)
#print(latest_file)

#Creates a dataframe of the Archived records
df_arc = pd.read_csv(latest_file, parse_dates=['Breach Submission Date'])
df_arc['DownloadDate'] = today
df_arc['FileType'] = fileformat.strip('_')

#print(df_arc.head())


202011060909
202011060910


In [2]:
#This step combines the two dataframes together for one completed file for visualization
df_comb= df_arc.append(df_ui)

#Following are some check steps I used to see the sizes of each of the dataframes used
#print(df_arc.shape)
#print(df_ui.shape)
#print(df_comb.shape)
#print(df_comb.info())

In [3]:
#This set of code writes the data to a google sheet which is the source file for a Tableau Public workbook

import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
scope = [
   'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
#Name of our Service Account Key
google_key_file = r'C:\Users\ted\PycharmProjects\GoogleSheet\black-works-294121-c065a334cd8d.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

spreadsheet_key = '1hrdiSg1CYvmI6-L38Z3B1H86zbFg0Kup8cdNguAjfL8'

wks_name = 'Sheet1'
#workbook = gc.open_by_key(spreadsheet_key)

sheet = gc.open("BreachSheet").sheet1


#Deletes all of the code in the existing sheet and resizes to create "fresh" worksheet.
sheet.clear()
sheet.resize(rows=1, cols=1)


#this last step sends the data to GoogleSheets
set_with_dataframe(sheet, df_comb, include_column_header=True)